In [53]:
import requests

import pandas as pd
from PIL import Image
import io
import numpy as np
np.random.seed(0)

plankton_names = [
        "aphanizomenon",
        "asplanchna",
        "asterionella",
        "bosmina",
        "brachionus",
        "ceratium",
        "chaoborus",
        "conochilus",
        "copepod_skins",
        "cyclops",
        "daphnia",
        "daphnia_skins",
        "diaphanosoma",
        "diatom_chain",
        "dinobryon",
        "dirt",
        "eudiaptomus",
        "filament",
        "fish",
        "fragilaria",
        "hydra",
        "kellicottia",
        "keratella_cochlearis",
        "keratella_quadrata",
        "leptodora",
        "maybe_cyano",
        "nauplius",
        "paradileptus",
        "polyarthra",
        "rotifers",
        "synchaeta",
        "trichocerca",
        "unknown",
        "unknown_plankton",
        "uroglena",
    ]
def retrieve_images(plankton):
    repo_url_base = "https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/"
    photo_prefix = "SPC-EAWAG-0P5X-"
    file_suffix = ".jpeg"
    raw_suffix = "?raw=True"
    imgs = []
    repo_data = repo_url_base + plankton + "/training_data/"
    print("Retrieving..." + repo_data)
    r = requests.get(repo_data)
    for o in str(r.content).split(photo_prefix)[1:49]:
        img_url = (
            repo_data
            + photo_prefix
            + o.split(file_suffix)[0]
            + file_suffix
            + raw_suffix
        )
        print("Image@", img_url)
        img = requests.get(img_url)
        x = Image.open(io.BytesIO(img.content))
        imgs.append(x)
    return imgs

In [7]:
# Simple helper to assign data
def split_np_array_train_test(arr, train_test_ratio:float = 0.75):
    split = int(round(len(arr)*train_test_ratio,0))
    np.random.shuffle(arr)
    return (arr[:split],arr[split:],)

In [8]:
# Simple helper to resize images bilinearly
import math
def bilinear_interpolation(image, y, x):
    height = image.shape[0]
    width = image.shape[1]

    x1 = max(min(math.floor(x), width - 1), 0)
    y1 = max(min(math.floor(y), height - 1), 0)
    x2 = max(min(math.ceil(x), width - 1), 0)
    y2 = max(min(math.ceil(y), height - 1), 0)

    a = float(image[y1, x1])
    b = float(image[y2, x1])
    c = float(image[y1, x2])
    d = float(image[y2, x2])

    dx = x - x1
    dy = y - y1

    new_pixel = a * (1 - dx) * (1 - dy)
    new_pixel += b * dy * (1 - dx)
    new_pixel += c * dx * (1 - dy)
    new_pixel += d * dx * dy
    return round(new_pixel)


def bl_resize(image, new_height, new_width):
    new_image = np.zeros((new_height, new_width), image.dtype)  # new_image = [[0 for _ in range(new_width)] for _ in range(new_height)]

    orig_height = image.shape[0]
    orig_width = image.shape[1]

    # Compute center column and center row
    x_orig_center = (orig_width-1) / 2
    y_orig_center = (orig_height-1) / 2

    # Compute center of resized image
    x_scaled_center = (new_width-1) / 2
    y_scaled_center = (new_height-1) / 2

    # Compute the scale in both axes
    scale_x = orig_width / new_width;
    scale_y = orig_height / new_height;

    for y in range(new_height):
        for x in range(new_width):
            x_ = (x - x_scaled_center) * scale_x + x_orig_center
            y_ = (y - y_scaled_center) * scale_y + y_orig_center
            new_image[y, x] = bilinear_interpolation(image, y_, x_)
    return new_image

In [74]:
from IPython import display
qubit_dims = (16,16,)
p0 = plankton_names[0]
def preprocess_images(image):
    x = image.convert("L") #grayscale
    x = np.asarray(x) # numpy array
    x = bl_resize(x, *qubit_dims) #resize
    x = x / 255.0 #normalize
    return x
p0_ims, p1_ims = retrieve_images(plankton_names[0]),retrieve_images(plankton_names[1])
ims_0, ims_1 = [preprocess_images(i) for i in p0_ims], [preprocess_images(i) for i in p1_ims]

Retrieving...https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543372901157-3725350526242-001629-055-1224-2176-84-64.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphanizomenon/training_data/SPC-EAWAG-0P5X-1570543374882008-3725352526408-001649-138-1826-1500-72-56.jpeg?raw=True
Image@ https://github.com/juleshenry/quantum-mnist/blob/main/data/zooplankton_0p5x/aphaniz

In [75]:
def make_test_train(ims: list, p: str, split_ratio:float=.75):
  split = int(len(ims)*split_ratio)
  x_train, x_test = ims[:split], ims[split:] 
  y_train, y_test = np.array([p]*len(x_train)), np.array([p]*len(x_test))
  return x_train,x_test,y_train,y_test
def shuffle(x,y):
  shuffle_ix = np.random.permutation(len(x))
  x = np.array([x[i] for i in shuffle_ix])
  y = np.array([y[i] for i in shuffle_ix])
  return x, y
data0 = make_test_train(ims_0,0)
data1 = make_test_train(ims_1,1)
x_train = np.array(data0[0] + data1[0])
x_test = np.array(data0[1] + data1[1])
y_train = np.append(data0[2] , data1[2])
y_test = np.append(data0[3] , data1[3])
x_train, y_train = shuffle(x_train, y_train)
x_test, y_test = shuffle(x_test, y_test)
print([len(o) for o in [x_train,x_test,y_train,y_test]])

[72, 24, 72, 24]


In [77]:
import tensorflow as tf
def create_fair_classical_model():
    # A simple model based off LeNet from https://keras.io/examples/mnist_cnn/
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(*qubit_dims,1)))
    model.add(tf.keras.layers.Dense(2, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    return model


model = create_fair_classical_model()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [80]:
model.fit(x_train,
          y_train,
          batch_size=16,
          epochs=20,
          verbose=2,
          validation_data=(x_test, y_test))

fair_nn_results = model.evaluate(x_test, y_test)

Epoch 1/20
5/5 - 0s - loss: 0.5923 - accuracy: 0.7917 - val_loss: 0.6442 - val_accuracy: 0.6250 - 169ms/epoch - 34ms/step
Epoch 2/20
5/5 - 0s - loss: 0.5880 - accuracy: 0.8611 - val_loss: 0.6430 - val_accuracy: 0.6250 - 91ms/epoch - 18ms/step
Epoch 3/20
5/5 - 0s - loss: 0.5845 - accuracy: 0.8750 - val_loss: 0.6417 - val_accuracy: 0.6250 - 36ms/epoch - 7ms/step
Epoch 4/20
5/5 - 0s - loss: 0.5804 - accuracy: 0.9167 - val_loss: 0.6405 - val_accuracy: 0.6250 - 38ms/epoch - 8ms/step
Epoch 5/20
5/5 - 0s - loss: 0.5767 - accuracy: 0.9167 - val_loss: 0.6394 - val_accuracy: 0.7500 - 38ms/epoch - 8ms/step
Epoch 6/20
5/5 - 0s - loss: 0.5728 - accuracy: 0.9167 - val_loss: 0.6382 - val_accuracy: 0.7500 - 34ms/epoch - 7ms/step
Epoch 7/20
5/5 - 0s - loss: 0.5688 - accuracy: 0.9167 - val_loss: 0.6370 - val_accuracy: 0.7500 - 33ms/epoch - 7ms/step
Epoch 8/20
5/5 - 0s - loss: 0.5649 - accuracy: 0.9167 - val_loss: 0.6360 - val_accuracy: 0.7500 - 42ms/epoch - 8ms/step
Epoch 9/20
5/5 - 0s - loss: 0.5609 - 